In [36]:
import os 

In [37]:
%pwd

'd:\\Udemy\\ML_Bootcamp\\18_End_to_End_DataScience_Project'

In [38]:
os.chdir("../")
%pwd

'd:\\Udemy\\ML_Bootcamp'

In [39]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass
class DataIngestionconfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
    


In [40]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_dictionaries

In [41]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH
                 ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_dictionaries([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config= self.config.data_ingestion
        create_dictionaries([config.root_dir])        
        
        data_ingestion_config = DataIngestionconfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config
               

In [42]:
import os 
import urllib.request as request
from src.datascience import logger
import zipfile

In [43]:
## component Data Ingestion

class DataIngestion:
    def __init__(self, config: DataIngestionconfig):
        self.config = config
    
    # Downloading the zip file
      
    def download_file(self):
        if os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file,
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists.")
    
    # Unzipping the downloaded file
    
    def extract_zip_file(self):
        """
        zip_file_path: str 
        Extract the zip file into the data directory
        
        Function returns None
        
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Zip file {self.config.local_data_file} extracted to {unzip_path}.")

In [45]:
try:
    conf= ConfigurationManager()
    data_ingestion_config=conf.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'